<a href="https://colab.research.google.com/github/HanudeepSetti/HanudeepSetti/blob/main/Ad%20Traffic%20Integrity%20Analysis%3A%20Detecting%20and%20Understanding%20IVT%20(Invalid%20Traffic)%20Patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import os

# --- File groups ---
valid_files = [
    "Data Analytics Assignment - Valid 1.csv",
    "Data Analytics Assignment - Valid 2.csv",
    "Data Analytics Assignment - Valid 3.csv"
]

invalid_files = [
    "Data Analytics Assignment - Invalid 1.csv",
    "Data Analytics Assignment - Invalid 2.csv",
    "Data Analytics Assignment - Invalid 3.csv"
]

# --- Empty lists for each data type ---
all_total, all_daily, all_hourly = [], [], []


# --- Help's function to extract Total, Daily, Hourly sections ---
def extract_sections(df_raw, app_name, ivt_value):
    df_raw = df_raw.applymap(lambda x: str(x).strip() if pd.notna(x) else x)

    def find_index(keyword):
        matches = df_raw.index[df_raw.iloc[:, 1].astype(str).str.contains(keyword, na=False, case=False)].tolist()
        return matches[0] if matches else None

    total_start = find_index("Total Data")
    daily_start = find_index("Daily Data")
    hourly_start = find_index("Hourly Data")

    if not total_start:
        print(f"Skipped {app_name}: 'Total Data' not found")
        return None, None, None

    total_end = daily_start - 1 if daily_start else None
    daily_end = hourly_start - 1 if hourly_start else None

    def clean_section(start, end, label):
        if start is None:
            return pd.DataFrame()
        try:
            section = df_raw.iloc[start+1 : end].copy() if end else df_raw.iloc[start+1 :].copy()
            section.columns = df_raw.iloc[start+1].values
            section = section.iloc[1:].dropna(how="all")
            section["data_type"] = label
            section["app_name"] = app_name
            section["IVT"] = ivt_value
            section.reset_index(drop=True, inplace=True)
            section.dropna(axis=1, how="all", inplace=True)
            return section
        except Exception as e:
            print(f"Error in {app_name} - {label} section:", e)
            return pd.DataFrame()

    df_total = clean_section(total_start, total_end, "Total")
    df_daily = clean_section(daily_start, daily_end, "Daily")
    df_hourly = clean_section(hourly_start, None, "Hourly")

    print(f"Processed {app_name}:",
          f"Total({df_total.shape}), Daily({df_daily.shape}), Hourly({df_hourly.shape})")

    return df_total, df_daily, df_hourly


# --- Common loop for both Valid and Invalid files ---
def process_file_group(file_list, ivt_value, prefix):
    for file in file_list:
        if not os.path.exists(file):
            print(f" Missing file: {file}")
            continue
        app_name = file.split(" - ")[-1].replace(".csv", "").strip()
        df_raw = pd.read_csv(file, header=None)
        df_total, df_daily, df_hourly = extract_sections(df_raw, f"App-{prefix}-{app_name}", ivt_value)
        if df_total is not None:
            all_total.append(df_total)
        if df_daily is not None:
            all_daily.append(df_daily)
        if df_hourly is not None:
            all_hourly.append(df_hourly)


# --- Process all datasets ---
process_file_group(valid_files, ivt_value=0, prefix="Valid")
process_file_group(invalid_files, ivt_value=1, prefix="Invalid")


# --- Combine all datasets ---
final_total = pd.concat(all_total, ignore_index=True)
final_daily = pd.concat(all_daily, ignore_index=True)
final_hourly = pd.concat(all_hourly, ignore_index=True)


# --- Data cleanup ---
def clean_dataframe(df):
    df.columns = [str(col).strip() for col in df.columns]
    df = df.loc[:, ~df.columns.duplicated()]
    for col in df.columns:
        if col not in ["data_type", "app_name"]:
            df[col] = pd.to_numeric(df[col], errors="ignore")
    df.fillna({"IVT": 0, "app_name": "Unknown", "data_type": "Unknown"}, inplace=True)
    return df


final_total = clean_dataframe(final_total)
final_daily = clean_dataframe(final_daily)
final_daily['Date'] = pd.to_datetime(final_daily['Date'], format='%Y-%m-%d %H:%M:%S')
final_hourly = clean_dataframe(final_hourly)
final_hourly['Date'] = pd.to_datetime(final_hourly['Date'], format='%Y-%m-%d %H:%M:%S')

print("Total:", final_total.head(10))
print("Daily:", final_daily.info())
print("Hourly:", final_hourly.info())


In [ ]:
# ---Checking for Null Values---
final_total.isna().sum()
final_daily.isna().sum()
final_hourly.isna().sum()

In [ ]:
#---Key analysis step---

from plotly.subplots import make_subplots

# Combine all datasets for unified analysis
ivt_data = pd.concat([
    final_total.assign(Level="Total"),
    final_daily.assign(Level="Daily"),
    final_hourly.assign(Level="Hourly")
], ignore_index=True)


sns.pairplot(ivt_data[['total_requests', 'impressions', 'requests_per_idfa', 'impressions_per_idfa', 'IVT']],
             hue='IVT', palette='coolwarm')
plt.suptitle("Pairplot: Requests/Impressions vs IVT", y=1.02)
plt.show()


In [ ]:
fig = px.line(final_daily, x='Date', y='total_requests', color='app_name',
              facet_col='IVT', title="App Traffic Over Time (Valid vs IVT)")
fig.update_layout(xaxis_title="Date", yaxis_title="Total Requests")
fig.show()


In [ ]:
fig = px.scatter_3d(final_daily, x="idfa_ip_ratio", y="requests_per_idfa",z="Date",
	         size="idfa_ua_ratio", color="IVT",
                 hover_name="Date", log_x=True, size_max=60)
fig.show()

In [ ]:
if 'Date' in final_daily.columns:
    daily_trend = final_daily.groupby(['Date', 'IVT'])['unique_idfas'].sum().reset_index()
    fig = px.line(daily_trend, x='Date', y='unique_idfas', color='IVT',
                  title="Daily unique idfas Trend (Valid vs IVT)",
                  color_discrete_map={0: 'blue', 1: 'red'})
    fig.update_layout(xaxis_title="Date", yaxis_title="Unique IDFA Count")
    fig.show()


In [ ]:
corr_valid = final_total[final_total['IVT']==0].corr(numeric_only=True)
corr_ivt = final_total[final_total['IVT']==1].corr(numeric_only=True)

fig, axes = plt.subplots(1, 2, figsize=(14,6))
sns.heatmap(corr_valid, ax=axes[0], cmap='coolwarm', annot=False)
axes[0].set_title("Valid Traffic Correlations")
sns.heatmap(corr_ivt, ax=axes[1], cmap='coolwarm', annot=False)
axes[1].set_title("IVT Traffic Correlations")
plt.show()


In [ ]:
ratio_cols = ['idfa_ip_ratio', 'idfa_ua_ratio', 'requests_per_idfa', 'impressions_per_idfa']
for col in ratio_cols:
    fig = px.violin(final_hourly, x='IVT', y=col, color='app_name', box=True, points="all",
                    title=f"{col} Distribution by IVT Type")
    fig.show()


In [ ]:
# Filter out rows where impressions are zero to avoid ZeroDivisionError

fig = px.sunburst(final_daily, path=['app_name',"unique_idfas","idfa_ip_ratio"], values="unique_uas",
                  color='app_name', color_discrete_map={0:'blue', 1:'red'},
                  title="Traffic Composition (App → IVT → Level)")
fig.show()

In [ ]:
fig = px.density_heatmap(final_hourly, x='idfa_ip_ratio', y='requests_per_idfa',
                         facet_col='IVT', title="Traffic Density by IVT Type")
fig.show()
